In [1]:
import pandas as pd
# import tensorflow
# import datasets
from transformers import Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils import data
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
from tqdm import tqdm
import wandb
#import os

# wandb.init(project="my-test-project", entity="effendi")

In [2]:
train_data = pd.read_csv('data\si630w22-hw3-train.csv')
dev_data = pd.read_csv('data\si630w22-hw3-dev.csv')
q_and_a_data = pd.read_csv('data\si630w22-hw3-data.csv')

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17845 entries, 0 to 17844
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            17845 non-null  object 
 1   annotator_id  17845 non-null  object 
 2   rating        17842 non-null  float64
 3   group         17845 non-null  object 
dtypes: float64(1), object(3)
memory usage: 557.8+ KB


In [4]:
dev_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            3810 non-null   object 
 1   annotator_id  3810 non-null   object 
 2   rating        3809 non-null   float64
 3   group         3810 non-null   object 
dtypes: float64(1), object(3)
memory usage: 119.2+ KB


In [5]:
train_data.rename(columns={'id':'question_id'}, inplace = True)

In [6]:
q_and_a_data

,question_id,question_text,reply_id,reply_text,rlen
0,t3_n27vu3,What's something nice you like to do just to b...,gwhrhmf,Give compliments. It’s extremely easy to do an...,205
1,t3_n2az7m,So what is the best headphones for people who ...,gwiatps,I prefer Raycon Performance Ear Buds. They are...,178
2,t3_n2dzr9,How do you go on knowing a loved one only has ...,gwit1wj,Make it as memorable as the rest of your time ...,278
3,t3_n2iy9q,You’ve been dropped to the year 1800 with all ...,gwjhw8i,They're gonna burn me at the stake for being a...,52
4,t3_n2kuuq,Stuck in bad habits for years and I realized i...,gwkiiie,Using new environments as a way to create heal...,651
...,...,...,...,...,...
5395,t3_njln4h,What is the first thing that comes to mind whe...,gz82i2v,"Austin, we have a problem. Just watched an epi...",168
5396,t3_nkx9c8,What career advice would you give to someone w...,gzfan0b,Work anywhere while you build qualifications a...,449
5397,t3_nm2mc0,What was your most unexplainable and bizarre e...,gzm4krx,Doing my homework at the desk when I was like ...,348
5398,t3_nnbl3i,Do other countries have as much coverage of UF...,gztlr9g,"In Mexico, there’s barely any news if any but ...",377


In [7]:
# connect the q and answer par using [SEP] token
q_and_a_data['combined'] = q_and_a_data.question_text + '[SEP]' + q_and_a_data.reply_text

In [8]:
pd.options.display.max_colwidth = 100
q_and_a_data.combined.head(1)

0    What's something nice you like to do just to be nice?[SEP]Give compliments. It’s extremely easy ...
Name: combined, dtype: object

In [9]:
merged_data = pd.merge(q_and_a_data, train_data, on='question_id')

In [10]:
merged_data.info()
# remove the rows that do not have a rating

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17845 entries, 0 to 17844
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   question_id    17845 non-null  object 
 1   question_text  17845 non-null  object 
 2   reply_id       17845 non-null  object 
 3   reply_text     17845 non-null  object 
 4   rlen           17845 non-null  int64  
 5   combined       17845 non-null  object 
 6   annotator_id   17845 non-null  object 
 7   rating         17842 non-null  float64
 8   group          17845 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 1.4+ MB


In [11]:
from transformers import AutoTokenizer, AutoModel
  
tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
model = AutoModel.from_pretrained("microsoft/MiniLM-L12-H384-uncased")

inputs = tokenizer("Hello world!", return_tensors="pt")
outputs = model(**inputs)

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/127M [00:00<?, ?B/s]

In [13]:
inputs

{'input_ids': tensor([[ 101, 7592, 2088,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [29]:
merged_data_test = merged_data.combined.copy().values
input_dict = {}
i = 0
for line in merged_data_test:
    print(line)
    tokenized_line = tokenizer(line, padding = 'max_length', truncation=True, max_length = 512,return_tensors="pt") #return_tensors is key
    print(tokenized_line)
#     input_line = {}
#     for k,v in tokenized_line.items():
#         input_line[k] = torch.Tensor(v) 
#     #print(input_line)
#     input_dict[i] = input_line
#     i += 1
    break
#print(input_dict)

What's something nice you like to do just to be nice?[SEP]Give compliments. It’s extremely easy to do and could change someone’s day. Gotta be careful to do so in a way that doesn’t allow the other party to think you’re hitting on them, though. So it’s a balance.
{'input_ids': tensor([[  101,  2054,  1005,  1055,  2242,  3835,  2017,  2066,  2000,  2079,
          2074,  2000,  2022,  3835,  1029,   102,  2507, 19394,  2015,  1012,
          2009,  1521,  1055,  5186,  3733,  2000,  2079,  1998,  2071,  2689,
          2619,  1521,  1055,  2154,  1012, 10657,  2022,  6176,  2000,  2079,
          2061,  1999,  1037,  2126,  2008,  2987,  1521,  1056,  3499,  1996,
          2060,  2283,  2000,  2228,  2017,  1521,  2128,  7294,  2006,  2068,
          1010,  2295,  1012,  2061,  2009,  1521,  1055,  1037,  5703,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           

In [14]:
# tokenize the whole input
merged_data['tf_input'] = merged_data.combined.apply(lambda x: tokenizer(x, padding = 'max_length', truncation=True, max_length = 512))

AttributeError: 'Series' object has no attribute 'applymap'

In [16]:
merged_data['tf_input'].iloc[0]
# dataset = merged_data.combined.map(lambda e: tokenizer(e, truncation=True, padding='max_length', max_length = 512))
# dataset

{'input_ids': [101, 2054, 1005, 1055, 2242, 3835, 2017, 2066, 2000, 2079, 2074, 2000, 2022, 3835, 1029, 102, 2507, 19394, 2015, 1012, 2009, 1521, 1055, 5186, 3733, 2000, 2079, 1998, 2071, 2689, 2619, 1521, 1055, 2154, 1012, 10657, 2022, 6176, 2000, 2079, 2061, 1999, 1037, 2126, 2008, 2987, 1521, 1056, 3499, 1996, 2060, 2283, 2000, 2228, 2017, 1521, 2128, 7294, 2006, 2068, 1010, 2295, 1012, 2061, 2009, 1521, 1055, 1037, 5703, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [17]:
# dataset.iloc[0]

In [18]:
merged_data[['attention_mask','input_ids','token_type_ids']] = pd.DataFrame(merged_data.tf_input.tolist(), index= merged_data.index)

In [19]:
# df2 = pd.DataFrame(df['meta'].str.split().values.tolist())
merged_data.rename(columns={'rating':'label'}, inplace = True)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17845 entries, 0 to 17844
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   question_id     17845 non-null  object 
 1   question_text   17845 non-null  object 
 2   reply_id        17845 non-null  object 
 3   reply_text      17845 non-null  object 
 4   rlen            17845 non-null  int64  
 5   combined        17845 non-null  object 
 6   annotator_id    17845 non-null  object 
 7   label           17842 non-null  float64
 8   group           17845 non-null  object 
 9   tf_input        17845 non-null  object 
 10  attention_mask  17845 non-null  object 
 11  input_ids       17845 non-null  object 
 12  token_type_ids  17845 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.9+ MB


In [44]:
# type(tf_input.head(1))
# tf_input.head(1)[0]
merged_data.input_ids.head(2)

0    [101.0, 2054.0, 1005.0, 1055.0, 2242.0, 3835.0, 2017.0, 2066.0, 2000.0, 2079.0, 2074.0, 2000.0, ...
1    [101.0, 2054.0, 1005.0, 1055.0, 2242.0, 3835.0, 2017.0, 2066.0, 2000.0, 2079.0, 2074.0, 2000.0, ...
Name: input_ids, dtype: object

In [40]:
merged_data['input_ids'] = merged_data['input_ids'].apply(lambda a: [float(x) for x in a])
merged_data['token_type_ids'] = merged_data['token_type_ids'].apply(lambda a: [float(x) for x in a])
merged_data['attention_mask'] = merged_data['attention_mask'].apply(lambda a: [float(x) for x in a])

In [43]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17845 entries, 0 to 17844
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   question_id     17845 non-null  object 
 1   question_text   17845 non-null  object 
 2   reply_id        17845 non-null  object 
 3   reply_text      17845 non-null  object 
 4   rlen            17845 non-null  int64  
 5   combined        17845 non-null  object 
 6   annotator_id    17845 non-null  object 
 7   label           17842 non-null  float64
 8   group           17845 non-null  object 
 9   tf_input        17845 non-null  object 
 10  attention_mask  17845 non-null  object 
 11  input_ids       17845 non-null  object 
 12  token_type_ids  17845 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.9+ MB


In [32]:
merged_data.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

AttributeError: 'DataFrame' object has no attribute 'set_format'

In [52]:
class MyDataset(Dataset):

  def __init__(self,df_name):
#     print(df_name)
    x= df_name.values#
    print(x.iloc[0])
#     y= df_name.values

    self.x_train=torch.tensor(x,dtype=torch.float32)
#     self.y_train=torch.tensor(y,dtype=torch.float32)

  def __len__(self):
    return len(self.y_train)
  
  def __getitem__(self,idx):
    return self.x_train[idx]

In [80]:
target = merged_data[['input_ids', 'token_type_ids', 'attention_mask', 'label']].copy()
target.dropna(subset=['label'], inplace = True)
target['input_ids'] = target['input_ids'].apply(lambda a: [float(x) for x in a])
target['token_type_ids'] = target['token_type_ids'].apply(lambda a: [float(x) for x in a])
target['attention_mask'] = target['attention_mask'].apply(lambda a: [float(x) for x in a])
target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17842 entries, 0 to 17844
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   input_ids       17842 non-null  object 
 1   token_type_ids  17842 non-null  object 
 2   attention_mask  17842 non-null  object 
 3   label           17842 non-null  float64
dtypes: float64(1), object(3)
memory usage: 697.0+ KB


In [78]:
# x = target['input_ids'].iloc[0]
target['input_ids'] =target['input_ids'].apply(lambda x: torch.tensor(x,dtype=torch.float32))
target['token_type_ids'] =target['token_type_ids'].apply(lambda x: torch.tensor(x,dtype=torch.float32))
target['attention_mask'] =target['attention_mask'].apply(lambda x: torch.tensor(x,dtype=torch.float32))
target['label'] =target['label'].apply(lambda x: torch.tensor(x,dtype=torch.float32))

In [85]:
x = target['input_ids'].values
torch.tensor(x,dtype=torch.float32)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [77]:
TensorDataset(target[['input_ids', 'token_type_ids', 'attention_mask']], target['label'])

TypeError: 'numpy.int64' object is not callable

In [73]:
# target['input_ids']
from operator import itemgetter
# sorted(target.values.tolist(), key=itemgetter(1))
# dict(target.values.tolist())
target[['input_ids', 'token_type_ids', 'attention_mask']].values.tolist()

ValueError: dictionary update sequence element #0 has length 3; 2 is required

In [61]:
train_tensor = data.TensorDataset(target, target['label']) 
# train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

TypeError: 'numpy.int64' object is not callable

In [47]:
inputs

{'input_ids': tensor([[ 101, 7592, 2088,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

Input is a dictionary of the 4 different tensors.

In [53]:
MyDataset(target).dtype
# target.dtype

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [62]:
final = target.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

AttributeError: 'DataFrame' object has no attribute 'set_format'

In [91]:
# merged_data['input_ids'] = pd.to_numeric(merged_data['input_ids'])

# merged_data[['input_ids', 'token_type_ids', 'attention_mask']].apply(pd.to_numeric)

# tensor_ = torch.from_numpy(target.to_numpy().astype(np.float32))
# target

TypeError: Invalid object type at position 0

In [37]:
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
# tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

# define a function that will tokenize the model, and will return the relevant inputs for the model
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = 'max_length', truncation=True, max_length = 512)

# train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data))
# test_data = test_data.map(tokenization, batched = True, batch_size = len(test_data))

# train_data.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
train = data_utils.TensorDataset(features, target)
merged_data.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
# test_data.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

AttributeError: 'DataFrame' object has no attribute 'set_format'

In [22]:
tf_train_set = lm_dataset["train"].to_tf_dataset(columns=["attention_mask", "input_ids", "labels"])


{'input_ids': tensor([[ 101, 7592, 2088,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [ ]:
# model_checkpoint = "distilroberta-base"
# model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    "test-trainer",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
#     push_to_hub=True,
)

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


# instantiate the trainer class and check for available devices
# trainer = CustomTrainer(
#     model=model,
#     args=training_args,
#     tokenizer = tokenizer,
# #     compute_metrics=compute_metrics,
#     train_dataset=train_data,
#     eval_dataset=dev_data
# )
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_data,
    eval_dataset=dev_data,
#     data_collator=data_collator,
    tokenizer = tokenizer,
#     compute_metrics=compute_metrics,
)
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")